# Configuration

In [22]:
import os
import pandas as pd
#import seaborn            as sns
import spotipy
import spotipy.oauth2     as oauth2

from pprint import pprint
import csv
import os


from matplotlib           import pyplot as plt 
from IPython.core.display import display, HTML

In [19]:
# -------------------------------------------
# Jupyter notebook Settings
# -------------------------------------------
display( HTML( '<style>.container { width:95% !important; }</style>' ) )
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option( 'display.expand_frame_repr', False )

# -------------------------------------------
# Plotting Settings
# -------------------------------------------
plt.rcParams['figure.figsize'] = [10,10]
plt.rcParams['font.size'] = 24
#sns.set()
#sns.set_style("whitegrid")
#sns.despine(right = True)
pd.options.display.float_format = '{:.2f}'.format

In [20]:
# Getting env variables
CLI_ID = %env CLI_ID
CLI_KEY = %env CLI_KEY

In [21]:
# ---------------------------------------------
# Spotify Settings 
# ---------------------------------------------
credentials = oauth2.SpotifyClientCredentials(
    client_id = CLI_ID, 
    client_secret = CLI_KEY)
token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

/mnt/c/Users/Marcelo/Documents/Repos/spotify-analysis/.env/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  import sys


# Getting Spotify Data

In [29]:
# header row for csv file 
csv_headers = ["url", "name", "artist", "explicit", "popularity", "duration_ms"]
# whether you want to overwrite existing files or not
OVERWRITE = True

def write_playlist(username, uri):
    '''
    Query the spotify API and receive the playlist information. If mode is 'nan' you can view this information data structure in its raw form.
    Obtain the list of tracks from the playlist information data structure and write it to a txt or csv file.
    Select a random song from the list of tracks and print general information to the console. 
    '''
    playlist_info = spotify.user_playlist(username, uri) 						#, fields='tracks,next,name'
    tracks = playlist_info['tracks']
    filename = "{0}.csv".format(playlist_info['name'])
    old_total = write_csv(filename, tracks)
    print("Number of tracks = {} --> {} ".format(old_total, tracks['total']))

def write_csv(filename, tracks):
	'''
	ADD TO CSV FILE
	View the playlist information data structure if this is confusing! 
	Specify the destination file path and check if the file exists already. If the file exists and you selected to not overwrite, the program will end here.
	Traverse the tracks data structure and add whatever information you want to store to a python list. These are the rows for your csv file
	Append all of these lists to a main python list which will store all the rows for your csv file.
	Write the data to the csv file!
	Exceptions handle the cases where the characters in the track info cannot be understood by the system and where the key is invalid (usually due to local files in the playlist).
	'''
	filepath = "./{0}}".format(filename)
	tracklist = []
	tracklist.append(csv_headers)
	if os.path.isfile(filepath):
		print("File already exists!")
		if not OVERWRITE:
			return
		else:
			print("Rewriting...")
	while True:
		for item in tracks['items']:
			if 'track' in item:
				track = item['track']
			else:
				track = item
			try:
				track_url = track['external_urls']['spotify']
				# add to list of lists
				track_info = [track_url, track['name'], track['artists'][0]['name'], track['explicit'], track['popularity'], track['duration_ms']]
				tracklist.append(track_info)
			except KeyError:
				print("Skipping track (LOCAL ONLY) - {0} by {1}".format(track['name'], track['artists'][0]['name']))
		if tracks['next']:
			tracks = spotify.next(tracks)
		else:
			break
	with open(filepath, 'w', newline='') as file:
		try:
			writer = csv.writer(file)
			writer.writerows(tracklist)
		except UnicodeEncodeError:
			print("Skipping track (UNDEFINED CHARACTERS) - {0} by {1}".format(track['name'], track['artists'][0]['name']))
	print("Playlist written to file.", end="\n\n")
	print("-----\t\t\t-----\t\t\t-----\n")
	return

write_playlist('marcelodof', '1BPIL9bSFtWI6SrSxGdWPn')

ValueError: Single '}' encountered in format string